In [87]:
import numpy as np
import pandas as pd 

from zytholic_project.reviews_data import BaseModelRev



In [58]:
dfrev = pd.read_csv("../raw_data/reviews_top_beer_concatenated.csv",nrows=100)


In [59]:
dfrev = dfrev.drop(columns='Unnamed: 0')



In [60]:
dfrev['beer_id'].nunique()



100

In [ ]:
#Regroup and join review by beer_id
#dfrev['score'] = dfrev.groupby(['beer_id'])['score'].transform(lambda x : np.mean(x))
#dfrev['look'] = dfrev.groupby(['beer_id'])['look'].transform(lambda x : np.mean(x))
#dfrev['smell'] = dfrev.groupby(['beer_id'])['smell'].transform(lambda x : np.mean(x))
#dfrev['taste'] = dfrev.groupby(['beer_id'])['taste'].transform(lambda x : np.mean(x))
#dfrev['feel'] = dfrev.groupby(['beer_id'])['feel'].transform(lambda x : np.mean(x))
#dfrev['overall'] = dfrev.groupby(['beer_id'])['overall'].transform(lambda x : np.mean(x))
#dfrev['text'] = dfrev.groupby(['beer_id'])['text'].transform(lambda x : ' '.join(x))

In [ ]:
#dfrev['score','look','smell','taste','feel','overall'] = dfrev.groupby(['beer_id'])['score','look','smell','taste','feel','overall'].transform(lambda x : np.mean(x))
#dfrev['text'] = dfrev.groupby(['beer_id'])['text'].transform(lambda x : ' '.join(x))


In [ ]:
#dfrevtext['text'] = dfrev.groupby(['beer_id'])['text'].transform(lambda x : ' '.join(x))
#dfrevtext

In [ ]:
#dfrevclean = dfrev.drop(columns=['username', 'date'])
#dfrevclean = dfrevclean[dfrev['text'] != '\xa0\xa0']
#dfrevtext.drop_duplicates(inplace = True)


#dfrevtext = dfrevtext.groupby(['beer_id'])

In [61]:
#dfrev = dfrev.groupby(['beer_id']).mean()
#dfrev.drop_duplicates(subset='text',inplace = True)
#dfrev.sort_values(by=['beer_id'])
#dfrevclean = dfrevclean[dfrev['text'] != '\xa0\xa0']

#Cleaning of the reviews Text

#dfrevclean['text'] = dfrevclean['text'].str.replace('\xa0','')
import string
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

def preprocessing(text):
    for punctuation in string.punctuation: 
        text = text.replace(punctuation, ' ') 
        
    lowercased = text.lower()
    
    clean = ''.join([i for i in lowercased if not i.isdigit()])
    
    replaced = clean.replace('\xa0','')
    
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(replaced)
    text = [w for w in word_tokens if not w in stop_words]
    
    lemmatizer = nltk.stem.WordNetLemmatizer()
    text = " ".join([lemmatizer.lemmatize(token) for token in text]).strip()
    
    garbage = "~`!@#$%^&*()_-+={[}]|\:;'<,>.?/"              
    text = "".join([char for char in text if char not in garbage])
    return text


dfrev['text'] = dfrev['text'].apply(preprocessing)

#text

In [82]:
dfrev




,beer_id,text
0,125646,oz funny story finally walked door min wait li...
1,58482,one pours cloudy rusty orange finger worth whi...
2,47658,pours black half finger khaki head last ring l...
3,173310,date april occasion draft upon u cleveland dra...
4,88605,release abv deep dark brown body basically opa...
...,...,...
95,67879,clear deep amber colour smell caramel cereal m...
96,68408,ml bottle poured snifter vintage appearance pa...
97,68680,thin dark pour worryingly small amount carbona...
98,68916,tasty orange forward well hopped balanced ipa ...


In [65]:
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english',token_pattern = '[a-z]+\w*',min_df=20)

# token_pattern = '[a-z]+\w*'

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(dfrev['text'])



#Output the shape of tfidf_matrix


tfidf_matrix.shape


(100, 2120)

In [ ]:
tfidf.get_feature_names()

In [66]:
feat = tfidf_matrix.toarray()
len(feat[1])


2120

In [67]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=10, n_iter=10, random_state=42)
svd.fit(tfidf_matrix)



TruncatedSVD(n_components=10, n_iter=10, random_state=42)

In [89]:
svd.explained_variance_ratio_
svd.singular_values_
feat = svd.transform(tfidf_matrix)
feat = pd.DataFrame(feat)




feat



,0,1,2,3,4,5,6,7,8,9
0,0.680746,-0.227496,-0.068139,-0.208773,0.322733,0.113152,0.109740,-0.001957,-0.161758,-0.015805
1,0.860665,-0.233396,0.043565,0.049276,0.134973,0.042670,-0.014528,0.187968,-0.064181,-0.036189
2,0.546582,0.553943,0.059207,-0.025193,0.182425,-0.254020,0.148979,-0.105162,-0.116839,0.150799
3,0.788596,-0.253321,-0.038617,-0.175826,0.212151,0.068142,0.032883,0.086018,-0.128071,-0.087607
4,0.565027,0.348367,0.512119,-0.311548,-0.056763,-0.094841,0.109543,0.071353,0.130602,-0.071311
...,...,...,...,...,...,...,...,...,...,...
95,0.585014,-0.053674,0.013845,-0.072461,-0.277682,0.014672,-0.176327,0.104206,-0.118182,-0.065841
96,0.660997,-0.133001,0.334573,0.273032,0.300186,0.164315,-0.173056,-0.216462,0.160798,-0.012647
97,0.742035,0.537091,-0.123325,0.019867,0.094680,-0.091096,-0.122385,0.031079,-0.015050,-0.061699
98,0.814292,-0.266395,-0.161179,-0.261137,0.233285,0.063426,0.109458,-0.068396,-0.035737,0.016368


In [83]:
result = pd.concat([dfrev, feat], axis=1, join="inner")


In [84]:
result


,beer_id,text,0,1,2,3,4,5,6,7,8,9
0,125646,oz funny story finally walked door min wait li...,0.680746,-0.227496,-0.068139,-0.208773,0.322733,0.113152,0.109740,-0.001957,-0.161758,-0.015805
1,58482,one pours cloudy rusty orange finger worth whi...,0.860665,-0.233396,0.043565,0.049276,0.134973,0.042670,-0.014528,0.187968,-0.064181,-0.036189
2,47658,pours black half finger khaki head last ring l...,0.546582,0.553943,0.059207,-0.025193,0.182425,-0.254020,0.148979,-0.105162,-0.116839,0.150799
3,173310,date april occasion draft upon u cleveland dra...,0.788596,-0.253321,-0.038617,-0.175826,0.212151,0.068142,0.032883,0.086018,-0.128071,-0.087607
4,88605,release abv deep dark brown body basically opa...,0.565027,0.348367,0.512119,-0.311548,-0.056763,-0.094841,0.109543,0.071353,0.130602,-0.071311
...,...,...,...,...,...,...,...,...,...,...,...,...
95,67879,clear deep amber colour smell caramel cereal m...,0.585014,-0.053674,0.013845,-0.072461,-0.277682,0.014672,-0.176327,0.104206,-0.118182,-0.065841
96,68408,ml bottle poured snifter vintage appearance pa...,0.660997,-0.133001,0.334573,0.273032,0.300186,0.164315,-0.173056,-0.216462,0.160798,-0.012647
97,68680,thin dark pour worryingly small amount carbona...,0.742035,0.537091,-0.123325,0.019867,0.094680,-0.091096,-0.122385,0.031079,-0.015050,-0.061699
98,68916,tasty orange forward well hopped balanced ipa ...,0.814292,-0.266395,-0.161179,-0.261137,0.233285,0.063426,0.109458,-0.068396,-0.035737,0.016368


In [29]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [30]:
cosine_sim.shape

(100, 100)

In [ ]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_beer = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_beer = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_beer = sim_beer[1:11]

    # Get the movie indices
    beer_id = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return metadata['title'].iloc[beer_id]

In [1]:
from zytholic_project.reviews_data import BaseModelRev

In [2]:
model = BaseModelRev()
model.get_data()

In [10]:
working_df = model.working_df
working_df.columns

Index(['name', 'original_style', 'brewery', 'abv', 'ave rating', 'min ibu',
       'max ibu', 'astringency', 'body', 'alcohol', 'bitter', 'sweet', 'sour',
       'salty', 'fruits', 'hoppy', 'spices', 'malty', 'beer_id', 'state',
       'country', 'retired', 'style', 'milk', 'old', 'dark', 'wild', 'pale',
       'red', 'imperial'],
      dtype='object')

In [ ]:
working_df = pd.merge(working_df, dfrev, how='inner', on=['beer_id'])
working_df = working_df.drop(columns='beer_id')
